# SVM

Instalando requisitos

In [1]:
%pip install -r requirements.txt -q
%pip install pandas -q
%pip install scikit-learn -q
%pip install tensorflow -q
%pip install numpy -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Importando requisitos

In [2]:
from app.modelo.imagem_rgb import ImagemRGB
import sklearn.model_selection
import pandas
import tensorflow
import os
import numpy
import pickle
import random
from sklearn import svm

2024-06-04 20:24:58.916897: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 20:25:09.193361: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Definindo variáveis

In [3]:
image_directory = os.path.join(os.getcwd(), 'dataset_converted')
num_classes = 6

Abrindo base de dados

In [4]:
data_frame = pandas.read_csv("classifications.csv")

Adicionando coluna path

In [5]:
for index, row in data_frame.iterrows():
    data_frame.at[index, 'path'] = os.path.join(image_directory, row['bethesda_system'], "{}.png".format(row['cell_id']))

In [6]:
data_frame

,image_id,image_filename,image_doi,cell_id,bethesda_system,nucleus_x,nucleus_y,path
0,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,1,SCC,792,462,/mnt/home/henrique/Repositorios_Git/GitHub.com...
1,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,2,SCC,601,678,/mnt/home/henrique/Repositorios_Git/GitHub.com...
2,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,3,SCC,363,467,/mnt/home/henrique/Repositorios_Git/GitHub.com...
3,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,4,SCC,599,437,/mnt/home/henrique/Repositorios_Git/GitHub.com...
4,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,5,Negative for intraepithelial lesion,1186,450,/mnt/home/henrique/Repositorios_Git/GitHub.com...
...,...,...,...,...,...,...,...,...
11529,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11530,LSIL,618,407,/mnt/home/henrique/Repositorios_Git/GitHub.com...
11530,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11531,LSIL,607,374,/mnt/home/henrique/Repositorios_Git/GitHub.com...
11531,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11532,LSIL,707,251,/mnt/home/henrique/Repositorios_Git/GitHub.com...
11532,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11533,LSIL,579,246,/mnt/home/henrique/Repositorios_Git/GitHub.com...


Adicionando coluna random e ordenando por meio dela

In [7]:
data_frame['random'] = data_frame[data_frame.columns[0]].apply(lambda _: random.random())

In [8]:
data_frame.sort_values(
    by='random',
    inplace=True
)

Removendo imagens que não existem

In [9]:
for index, row in data_frame.iterrows():
    if not os.path.exists(row['path']):
        data_frame.drop(index, inplace=True)

In [10]:
data_frame

,image_id,image_filename,image_doi,cell_id,bethesda_system,nucleus_x,nucleus_y,path,random
4014,248,a5ce47f6a10f543bf7fdac28b709d30d.png,10.6084/m9.figshare.12230393,4015,Negative for intraepithelial lesion,674,131,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000156
10318,35,bf7767fb55e9b4e77ea458f825f06e65.png,10.6084/m9.figshare.12229628,10319,LSIL,426,467,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000218
7417,111,e57d535d86c5898be4e897eaed05c8c6.png,10.6084/m9.figshare.12229901,7418,Negative for intraepithelial lesion,259,328,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000269
4625,226,199ea9d75d6266bcc1be87995de6a70a.png,10.6084/m9.figshare.12230315,4626,Negative for intraepithelial lesion,506,167,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000400
5694,160,fbb86fcc0948f02be36a1348d1046688.png,10.6084/m9.figshare.12230090,5695,Negative for intraepithelial lesion,272,672,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000404
...,...,...,...,...,...,...,...,...,...
6239,143,c01b36e0a384197ca9a2e6bcd2c552e1.png,10.6084/m9.figshare.12230039,6240,Negative for intraepithelial lesion,1065,246,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999131
7577,103,b0fddd5dbd774fcc23ddf897ee7bf26c.png,10.6084/m9.figshare.12229874,7578,Negative for intraepithelial lesion,860,974,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999137
4281,237,8e39307fc74e68a5f4a6ae66764331b4.png,10.6084/m9.figshare.12230357,4282,LSIL,687,656,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999247
10772,25,f981c4aac622d7cce71a81305dd38ade.png,10.6084/m9.figshare.12229598,10773,ASC-US,593,534,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999362


Separando o x

In [11]:
data_frame['path'].iloc[0]

'/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_06_01/dataset_converted/Negative for intraepithelial lesion/4015.png'

In [12]:
xi = ImagemRGB.from_file(data_frame['path'].iloc[0]).matriz.flatten()
xi

array([225, 192, 208, ..., 235, 224, 224], dtype=uint8)

In [14]:
xi.shape

(30000,)

In [23]:
x = []
for index, row in data_frame.iterrows():
    image = ImagemRGB.from_file(row['path'])
    x.append(image.matriz.flatten()/255)
x = numpy.array(x)

In [25]:
x.shape

(5581, 30000)

In [26]:
x

array([[0.88235294, 0.75294118, 0.81568627, ..., 0.92156863, 0.87843137,
        0.87843137],
       [0.47058824, 0.22352941, 0.20784314, ..., 0.4       , 0.41568627,
        0.30980392],
       [0.03921569, 0.07843137, 0.24313725, ..., 0.31764706, 0.56862745,
        0.61176471],
       ...,
       [0.6745098 , 0.44705882, 0.49411765, ..., 0.95686275, 0.96862745,
        0.96470588],
       [0.24313725, 0.14509804, 0.28235294, ..., 0.63137255, 0.69411765,
        0.7372549 ],
       [0.44313725, 0.28627451, 0.17647059, ..., 0.84313725, 0.86666667,
        0.85882353]])

Separando o y

In [49]:
y = data_frame.iloc[:, 4].values
#label_encoder = sklearn.preprocessing.LabelEncoder()
#y = label_encoder.fit_transform(y)

In [50]:
labels = list(set(data_frame.iloc[:, 4].values))

In [51]:
labels

['Negative for intraepithelial lesion',
 'SCC',
 'ASC-US',
 'HSIL',
 'LSIL',
 'ASC-H']

In [56]:
y = [[1 if label==lbl else 0 for lbl in labels] for label in y]

In [62]:
y = numpy.array(y)

In [64]:
y

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]])

In [65]:
type(y)

numpy.ndarray

In [69]:
y.shape

(5581, 6)

Separando o conjunto de treino e teste

In [70]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y)

In [71]:
y_train

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [72]:
x_train.shape

(4185, 30000)

In [73]:
y_train.shape

(4185, 6)

In [74]:
x_test.shape

(1396, 30000)

In [75]:
y_test.shape

(1396, 6)

Exportando x_test, x_train, y_test, y_train para um arquivo externo

In [76]:
#with open('data.pkl', mode='wb') as file:
    #pickle.dump([x_train, x_test, y_train, y_test], file)

# Aplicando o SVM

Construindo o modelo

In [77]:
modelo = svm.SVC()

Treinando o modelo

In [78]:
modelo.fit(x_train, y_train)

In [81]:
import joblib
joblib.dump(modelo, 'modelo.pkl')

['modelo.pkl']

In [82]:
with open('modelo.pkl', mode='wb') as file:
    pickle.dump(modelo, file)